In [199]:
import bs4
from bs4 import BeautifulSoup

import re
import random
from collections import defaultdict
from itertools import chain
import pandas as pd
import codecs
import json
import os
import lxml.html

In [200]:
data_dir= "../data/wikipedia/"

In [234]:
for file in os.listdir(data_dir):
    if file.endswith(".html"):
        date=file.split("/")[-1].replace(".html", "")
        date= date.replace("_February_", "-02-").replace("_March_", "-03-")
        filepath = os.path.join(data_dir, file)
        print(date)
        with codecs.open(filepath, "r") as file:
            data = file.read()
        
        entities_dict, news_dict, event_dict =parse_one_page_update(data, date)
        
        with open(f"../data/output/news_dict_{date}.json", "w") as file:
            json.dump(news_dict, file)
        
        with open(f"../data/output/entities_dict_{date}.json", "w") as file:
            json.dump(entities_dict, file)
        
        with open(f"../data/output/events_dict_{date}.json", "w") as file:
            json.dump(event_dict, file)
        
        print(event_id)

2022-02-27
['Armed conflicts and attacks', 'Business and economy', 'Disasters and accidents', 'Health and environment', 'International relations', 'Law and crime', 'Politics and elections', 'Science and technology', 'Sports']
[<Selector xpath='./ul' data='<ul><li><a href="/wiki/Russo-Ukrainia...'>, <Selector xpath='./ul' data='<ul><li><a href="/wiki/2022_Russian_i...'>, <Selector xpath='./ul' data='<ul><li><a href="/wiki/2022_Eastern_A...'>, <Selector xpath='./ul' data='<ul><li><a href="/wiki/COVID-19_pande...'>, <Selector xpath='./ul' data='<ul><li><a href="/wiki/Reactions_to_t...'>, <Selector xpath='./ul' data='<ul><li><a href="/wiki/Protests_again...'>, <Selector xpath='./ul' data='<ul><li><a href="/wiki/2022_Belarusia...'>, <Selector xpath='./ul' data='<ul><li><a href="/wiki/Reactions_to_t...'>, <Selector xpath='./ul' data='<ul><li><a href="/wiki/Reactions_to_t...'>]
9 9
category: Armed conflicts and attacks
leading events -> ['<li><a href="/wiki/Russo-Ukrainian_War" title="Russo-U

In [ ]:
with codecs.open("../data/wikipedia/2022_February_25.html", "r") as file:
    data = file.read()
entities_dict, news_dict, event_dict =parse_one_page_update(data, "2022-02-25")

In [4]:
from lxml.cssselect import CSSSelector
from lxml import etree
from scrapy.selector import Selector

In [231]:
# https://upg-dh.newtfire.org/explainXPath.html

def parse_one_page_update(data, date):
    keywords = ["Russo", "Russia", "Ukrain"]
    entities_dict=dict()
    news_dict= defaultdict(list)
    event_dict=defaultdict(dict)
    event_id = 0
    
    sl = Selector(text=data).xpath('//div[@class="current-events-content description"]')
    cats = sl.xpath("//b//text()").getall()
    # direct children from  div. 
    uls = sl.xpath("./ul")
    print(cats)
    print(uls)
    print(len(cats), len(uls))
    if date=="2022-02-25":
        uls.pop(5)
    assert len(cats)==len(uls)
    
    for cat, ul in zip(cats, uls):
        print("category:", cat) # category for the individual events.
        lis = ul.xpath("./li")
        # texts = ul.xpath("./li/a").getall()
        texts = ul.xpath("./li").getall()
        print("leading events ->", texts)
        
        idxes=[idx for idx, title in enumerate(texts) if any([x in title for x in keywords])]
        print(lis)
        
        li_uls = [[li.xpath("./ul") for li in lis][x] for x in idxes]
        leading_events= [texts[x] for x in idxes]
                              
        if len(li_uls)>0:
            for idx, el in enumerate(li_uls[0]):
                leading_event= leading_events[idx]
                leading_event= Selector(text=leading_event)
                print("leading event:", leading_event) # TODO get the entity.
                leading_event_title = leading_event.css("a::attr(title)").get()
                leading_event_href= leading_event.css("a::attr(href)").get()
                if leading_event_title not in entities_dict:
                    entities_dict[leading_event_title] = leading_event_href
                
                for el_li in el.css("li"):
                    entities = []
                    news = [] # paper: link
                    text_list = el_li.xpath("descendant::text()").getall()
                    
                    # get direct events
                    if "\n" not in text_list:
                        print(el_li)
                        event_text = "".join(text_list)
                        event_text = "".join(re.split("\(|\)|\[|\]", event_text)[::2])
                        print(event_text)
                        
                        ### check the links.
                        for al in el_li.xpath("./a"):
                            if al.css("a::attr(title)").get():
                                title = al.css("a::attr(title)").get()
                                href = al.css("a::attr(href)").get()
                                entities.append(title)
                                if title not in entities_dict:
                                    entities_dict[title]=href
                            else:
                                href= al.css("a::attr(href)").get()
                                # text = al.css("a::text").extract()
                                news.append(href)
                        print("entities", entities)
                        print("news", news)
                        print("leading event: ", leading_event)
                        news_dict[event_id]=news
                        event_dict[event_id]={
                            "cateogry": cat,
                            "event_text": event_text,
                            "main_event": leading_event_title,
                            "date":date, 
                            "news": news,
                            "entities": entities
                        }
                        event_id +=1
                        
                    print("*"*40)
    return entities_dict, news_dict, event_dict


In [183]:
def parse_one_html(data, date):
    soup = BeautifulSoup(data, "html.parser")
    div = soup.find("div", {"class":"current-events-content description"})
    entities_dict = defaultdict(dict)
    news = defaultdict(list)
    event_dict= defaultdict(dict)
    counter = 0 
    category_ =""
    
    for elem in div:

        if elem.name=="p":
            category=elem.get_text() 
            category_ = category.replace("\n", "")
        if elem.name=="ul":
            # for next_elem in elem.next_elements:
            #     print(next_elem)
            #     try:
            #         for el in next_elem.find("li"):
            #             print(el.get_text())
            #             print("====")
            #     except Exception:
            #         print(next_elem)
            
                # if next_elem.name=="li":
                #     event= next_elem.get_text()
                #     print(event)
                #     print("====")
            for el in elem.find("li"):
                events = el.get_text().split("\n")
                
                entities_ls = []
                leading_event = events[0]
                print(leading_event)
                if any([x in leading_event for x in ["Russo", "Ukrain", "Russia"]]):
                    for a in el.find_all("a", href=True):
                        href = a["href"]
                        try:
                            title = a["title"]
                            entities_dict[title]={"link":href, "category":category_
                                                 , "leading": leading_event}
                            entities_ls.append(title)
                        except Exception:
                            if href.startswith("https:"):
                                news[category_].append({
                                "link":href, "leading":leading_event
                                })


                    event_dict[counter]={
                                    "events":events, 
                                    "leading":leading_event,
                                    "entities": entities_ls, 
                                    "date":date,
                                    "category":category_
                                }
                    counter+=1
                    print("======"*100)
    return event_dict, entities_dict, news
    
